In [180]:
import math
from civilpy.general import units
from civilpy.structural.steel import W, MC
from termcolor import colored

# Workbook Layout

8 Sheets,

1. Global
1. Superstructure DC
1. Girder
1. Floor Beam
1. Deck Plate
1. Abut End Floor Beam
1. Floor Beam Prop
1. Floor Beam - Non Bracing - Void

# Inputs

## Global Definitions

Contains the various Design Inputs, and summaries of the various checks being
performed. # //TODO - Assumes precast ties?

In [181]:
# Global Input
floorbeam_spacing = 2.6 * units('ft')
girder_spacing = 20 * units('ft')
ballast_plate_spacing = 3 * units('ft')
diaphragm_spacing_max = 10.00 * units('ft')
railroad_gage = 4 * units('ft') + 8.5 * units('inch') # //TODO - Not AREMA
span_length = 52.00 * units('ft')
girder_length = 54 * units('ft')
floor_length = 58 * units('ft') # //TOOD - how is this determined?
F_y = 50000 * units('psi')
F_u = 65000 * units('psi')
E_steel = 29000000 * units('psi')
poisson_ratio = 0.30
unit_weight_steel = 490 * units('lbf/ft^3')
max_diaphragm_spacing = 10 * units('ft')
diaphragms = 'w16x89'
girder_quantity = 2
bracing_section = 'MC10x33.6'

# Tie Dimension
tie_length = 8.5 * units('ft')
tie_width = 8 * units('inch')
tie_depth = 7 * units('inch')
tie_design_width = 8 * units('ft') # Design width for distrib loading, length
max_tie_spacing = 24 * units('inch') # used for dead load

# Derived Inputs
ballast_plates_clear_space = girder_spacing - 2*ballast_plate_spacing
stop_plate_detail_width = (girder_spacing - ballast_plates_clear_space)/2

## Superstructure DC Definitions

The following values are defined in the 'Superstructure DC' sheet

In [182]:
# Unit Weights
ballast_unit_weight = 120 * units('lbf/ft^3')
tie_unit_weight = 150 * units('lbf/ft^3') # //TODO - is this AREMA? seems excessive
track_unit_weight = 200 * units('lbf/ft')  # AREMA 8-2.2.3.b
waterproofing_unit_weight = 2 * units('lbf/ft^2')      # //TODO - What's the source for this
steel_connection_cont = .1                 # 10% of weight of steel
asphalt_unit_weight = 150 * units('lbf/ft^3')          # //TODO - is this always needed?
timber_unit_weight = 60 * units('lbf/ft^3')            # //TODO - Not used, assumed concrete

# Girder
girder_quantity = 2                        # //TODO - Should reference Global

# Floor Beam
floorbeam_quantity = 19

# Floor Beam (End)
end_floorbeam_quantity = 2

# Diaphragm
diaphragm_quantity = 20

# Lateral Bracing
lateral_bracing_length = 32.8024  * units('ft')
lateral_bracing_quantity = 4               # //TODO - Verify these values (low)

# Floor
floor_quantity = 1

# Diagonal Stop Plate
dia_stop_pl_t = .5 * units('in')           # //TODO - Verify what these are

# //TODO - Verify what this formula is dependent on
dia_stop_pl_width = ((1*units('ft') + 10 * units('inch'))**2 + (1 * units.ft + (1+3/16) * units('inch')) ** 2)**.5
dia_stop_pl_quantity = 40

# Horizontal Upper FLoor Plate
horz_upper_floor_pl_t = 0.50 * units('in')

# //TODO - This formula doesn't match the description next to it (1'-1 3/16" + 3"), and appears to be incorrect
horz_upper_floor_pl_width = ((1 + 10/12 + 13/16/12) + 3/12) * units('ft')
# 1 * units('ft') + 10 * units('inch') + 3/16 * units('inch') + 3 * units('inch')
horz_upper_floor_pl_quantity = 10

# Waterproofing
waterproofing_quant = 1

# Asphaltic Plank
asphaltic_plank_t = 1 * units('in')
asphaltic_plank_quant = 1

# Ballast Under Ties
ballast_under_ties_t = 9 * units('in')
ballast_under_ties_quant = 1

# Ties
tie_material = 'Concrete'
tie_spacing = 1.5 * units('ft')

# Ballast at tie level
tie_level_ballast_quant = 1
tie_level_ballast_sloped_reduction = 2 * units('ft^2')
tie_level_ballast_include_pre_move_weight = False

# Rail
rail_quantity = 1

# Future Ballast Allowance
future_ballast = 12 * units('inch')  # //TODO - update to 30       # //TODO - should probably be global
future_ballast_quantity = 1                # //TODO - are the quantity=1 needed

# Floor Beam Bracket
fb_bracket_flange_t = 0.75 * units('inch')
fb_bracket_flange_web = 8.0 * units('inch')
fb_bracket_flange_len = 3.33 * units('foot')
fb_bracket_web_t = .5 * units('inch')
fb_bracket_web_width = 1.83 * units('ft')
fb_bracket_web_height = 2.75 * units('ft')
fb_bracket_quantity = 12

## Girder Definitions

```python
# //TODO - The following are labeled as inputs, but are dependent on other sheets
floorbeam_spacing = 'Global'!G8
girder_spacing = 'Global'!G9
ballast_plate_spacing = 'Global'!G10
structure_length = 'Global'!G14            # //TODO - Verify this in template
F_y = 'Global'!G17
E = 'Global'!G19
poisson_ration = 'Global'!G20
steel_unit_weight = 'Global'!$G$21
railroad_gauge = 'Global'!G13
girders = 'Global'!G24
```

In [183]:
E80_50_ft = 1901.80 * units('kip*ft') # AREMA 15 - Table 1-16, Loads are for 1 rail
E80_55_ft = 2233.10 * units('kip*ft') # //TODO - Translate Table to df
if span_length > 30 * units('ft'):
    impact = .35                      # AREMA 15 - Table 15-1-8 # //TODO - Copy # //TODO - labeled input, is derivative
else:
    pass

# Girder Geometrics
girder_web_height = 60 * units.inch
girder_web_thickness = 0.625 * units.inch
girder_flange_width = 20 * units.inch
girder_flange_thickness = 1.625 * units.inch

# Compression Flange Check
L_brace = 4 * floorbeam_spacing      # //TODO - Labeled as input, but is derivative

# Maxium Transverse stiffener clear distance
trans_stiff_da = 96 * units('inch')
trans_stiff_actual_da = 60 * units.inch

stiffener_width_bst = 6 * units('in')
stiffener_thickness_tst = 0.5 * units('in')

# Bearing stiffeners
bearing_stiffener_width_bsb = ((girder_flange_width - girder_web_thickness) / 2)   # AREMA 15- 1.7.7.a # //TODO - Labeled input, is derivative
bearing_stiffener_thickness_tsb = 1 * units('in')
bearing_stiffener_corner_clip = 1 * units('in')

# Bearing Stiffener Weld
bearing_stiffener_fillet_weld_leg = 0.3125 * units('in')

## Floor Beam Definitions

```python
# //TODO - The following are labeled as inputs, but are dependent on other values
floorbeam_spacing = 'Global'!G8
girder_spacing = 'Global'!G9
ballast_plate_spacing = 'Global'!G10
diaphragm_spacing_max = 'Global'!G12
railroad_gauge = 'Global'!G13
F_y = 'Global'!G17
F_u = 'Global'!G18
E_steel = 'Global'!G19

# Dead Load
diaphragm_name = 'Global'!$G$23
floorbeams_w = G108 # Table looking up beam properties from 'Floor Beam Prop' sheet (Just AISC database)
bracing_length = 'Superstructure DC'!E55/4
bracing_weight_ft = 'Superstructure DC'!E56

# Live Load
a = G9/2 - G12/2
```

In [184]:
floorbeam = W('W24x250')              # Lookup from steel table
end_floorbeam = W('W21X166')
diaphragm = W('W16X89')
lateral_bracing = MC('MC10x33.6')

# Dead Load
diaphragm_weight_ft = 61 * units('lbf/ft') # AISC Lookup
diaphragm_quant = 1                        # //TODO - Per FB?

bracing_quant= 4                           # //TODO - Per FB?

# Live Load                                # AREMA 15-1.3.4.2.3
fb_A = 100 * units('kip')                     # Alternative live load axel
fb_S = 5 * units('ft')
fb_a = girder_spacing/2 - railroad_gage/2
# Impact Load
ballasted_deck_reduction = .9              # AREMA 1.3.5.b

# Rocking Load
rocking_percent = .2                       # AREMA 1.3.5.d

# Wind force on Loaded Bridge
w = 300 * units('lbf/ft')                  # AREMA 1.3.7.a
h = 8 * units('ft')

# Req Section Properties
diaphragm_dia_hole = 1.0 * units('in')
diaphragm_num_holes = 4
lateral_dia_holes = 1.0 * units('in')
lateral_num_holes = 2

# End Shear
web_conn_girder_holes_num = 6
web_conn_girder_holes_dia = 1 * units('in')

# FLoor Beam Fatigue
assumed_mean_impact_perc = .35            # AREMA Table 15-1-8

## Deck Plate Definitions

```python
# //TODO - The following are labeled as inputs, but are dependent on other values
flange_width = MIN('Abut End FLoor Beam'!G129, 'Floor Beam'!G107)
clear_space_between_ballast_pls = 'Global'!G10
```

In [185]:
deck_plate_thickness = 0.75 * units('in')
min_ballast_below_tie = 6 * units('in')

# Live Load
axel_load = 100 * units('kips')

# Impact Load
reduction_fact = .9                        # AREMA 15.1.3.5

# Rocking Effect
wheel_load_percentage = .2

# Deck Plate Design Width
deck_plate_width = 12.00 * units('in') # //TODO - How is this defined?

## Abut End Floor Beam Definitions

```python
# //TODO - The following are labeled as inputs, but are dependent on other values
floorbeam_spacing ='Floor Beam-Non Bracing-Void'!G8
flooring_on_girder = G14/2+('Global'!G16-'Global'!G14)/2
girder_spacing = 'Floor Beam-Non Bracing-Void'!G9
clear_space_between_ballast_plates = G16-2*I17
diaphragm_spacing = 'Floor Beam-Non Bracing-Void'!G11
steel_unit_weight = 'Global'!$G$21
railroad_gauge = 'Global'!$G$13      # //TODO - Why defined twice here?

# Dead Load (DL)
floorbeam_w = G130
diaphragm_section = 'Global'!$G$23
end_fb_S = 5 * units('ft')
cl_rail_cl_girder_dist = G16/2 - G24/2

# Required Section Properties
# Values are AISC Table Lookup

# Bearing Stiffener
bearing_stiffener_width = ROUNDDOWN((G129-G127)/2*4,0)/4  #AREMA 15-1.7.7.a
```

In [186]:
# Geometrics
# railroad_gage = 4+8.5/12            # //TODO - AREMA has a value, this could also be linked
F_y = 50000 * units('psi')            # //TODO - Why different than Global?
F_u = 650000 * units('psi')           # //TODO - Why different
E = 29000000 * units('psi')           # //TODO - Why different

# Dead Load (DL)
diaphragm_weight_per_ft = 61 * units('lbf/ft')
end_diaphragm_quantity = 1  # //TODO - These aren't included in the deadload calcs, only uses int floorbeam diaphragms

# Live Load                                      # AREMA 15-1.3.4.2.3
axel_alternative_live_load = 100 * units('kips') # //TODO Mirrored from above?

# Impact Load
ballasted_deck_reduction = 0.9                   # AREMA 1.3.5.b

# Rocking Load
rocking_percent = .2                          # AREMA 1.3.5.d # //TODO - Defined twice

# Wind force on Loaded Bridge
w = 300 * units('lbf/ft')                        # AREMA 1.3.7.a
h = 8 * units('ft')

# Non-Bracing/Jacking Loads
jack_pt_offset = 3.75 * units('ft')

# Holes (laterals)
lateral_bracing_hole_dia = 1 * units('inch')
lateral_bracing_hole_quantity = 2

# Holes (Diaphragm)
diaphragm_hole_dia = 1.0 * units('in')
diaphragm_hole_quantity = 6

# End Shear
web_conn_girder_holes_dia = 1.00 * units('in')
web_conn_girder_holes_quantity = 6

# Floor Beam Fatigue
assumed_mean_impact_load_percent = .35        # AREMA Table 15-1-8

# Bearing Stiffeners
bearing_stiffener_t = 1.0 * units('in')
bearing_stiffener_corner_clip = 1.0 * units('in')

# Bearing Stiffener Weld                     # AREMA 15-1.7.7.a
bearing_stiff_weld_leg = 0.3125 * units('in')

## Floor Beam Prop Definitions

In [187]:
# Just a copy of AISC Shape Database

## Floor Beam-Non Bracing - Void

```python
# //TODO - The following are labeled as inputs, but are dependent on other values

floorbeam_spacing_d = 'Global'!G8
girder_spacing = 'Global'!G9
clear_space_between_ballast_plates = 'Global'!G10
diaphragm_spacing_max = 'Global'!G12
railroad_gauge_g = 'Global'!G13
F_y = 'Global'!G17
F_u = 'Global'!G18
E_steel = 'Global'!G19
steel_unit_weight = 'Global'!$G$21

# Live Load
a = G9/2 - G12/2

# Required Section Properties
# Steel Table Lookups
```

In [188]:
## //TODO - These Values are irrelevant right? That's why the sheet is 'Void'?
# floorbeams = 106 * units('lbf/ft')
# floorbeam_quantity = 1
# floor_plate_thickness = 0.75 * units('in')
# stop_plate_length = 4.5 * units('ft')
# stop_plate_thickness = 0.5 * units('in')
# stop_plate_quantity = 2
# wp_ballast_plate_length = 4 * units('ft')
# wp_ballast_plate_unit_weight = 2 * ('lbf/ft^2')
# ballast_depth = 12 * units('in')
# future_ballast_depth = 12 * units('in')
# ballast_density = 120 * units('lbf/ft^3')    # AREMA 1.3.2 Table 15-1-5
# track_work_w_restraining_rails = 300 * units('lbf/ft')
# track_work_w_rest_rails_quant = 1
# concrete_tie_length = 8 * units('ft')
# concrete_tie_width = 8 * units('in')
# concrete_tie_depth = 9 * units('in')
# tie_spacing = 20 * units('in')
# concrete_unit_weight = 150 * units('lbf/ft^3')
# concrete_tie_clear_space = 14 * units('ft')

## Live Load
# A = 100 * units('kip')
# S = 5 * units('ft')

## Impact Load
# ballast_deck_reduction = .9     # AREMA 1.3.5.b

## Rocking Load
# rocking_percentage = .2         # AREMA 1.3.5.d


## Wind force on Loaded Bridge
# w = 300 * units('lbf/ft')
# h = 8 * units('ft')

## Required Section Properties
# lateral_bracing_hole_dia = 1.0 * units('in')
# lateral_bracing_hole_quant = 2

# diaphragm_hole_dia = 1.0 * units('in')
# diaphragm_hole_quantity = 5

## End Shear
# web_conn_girder_hole_quantity = 6
# web_conn_girder_hole_dia = 1 * units('in')

## Bolted Connection
# bolt_dia = 0.875 * units('in')
# bolt_spacing = 2.75 * units('in')

## Floor Beam Fatigue
# assumed_mean_impact_load_percent = .35

## Cooper E80 Load Deflection
# A = 80 * units('kip')
# S = 5 * units('ft')

# Calcs

## Global

```python
# Design is adequate check
if girder or floorbeam_bracing or abutment_end_floorbeam or deck_plate == 'NG':
    adequacy = 'NG'
else:
    adequacy = 'OK'
```

```python
# Adequacy Checks
girder = 'Girder'!K8
floorbeam_bracing = 'Floor Beam'!L7
abutment_end_floorbeam = 'Abut End Floor Beam'!L7
deck_plate = 'Deck Plate'!M8
```

```python
# Governing Limit States
girder = 'Girder'!M8
floorbeam_bracing = 'Floor Beam-Non Bracing-Void'!N7 # //TODO - V
erify this ref
abutment_end_floorbeam = 'Abut End Floor Beam'!N7
deck_plate = 'Deck Plate'!O8
```

```python
# Governing D/C Ratio
girder = 'Girder'!L8
floorbeam_bracing = 'Floor Beam-Non Bracing-Void'!M7 # //TODO - Verify this ref
abutment_end_floorbeam = 'Abut End Floor Beam'!M7
deck_plate = 'Deck Plate'!N8
```

```python
# Section
girder = 'Plate girder'
floorbeam_bracing = 'Floor Beam'!G7
abutment_end_floorbeam = 'Abut End Floor Beam'!G7
deck_plate = ='Deck Plate'!E14 & CHAR(34) & " Thick PL."
```

## Preliminary Girder Calcs

This was moved ahead of DC Calcs because later calcs depend on the results here

In [189]:
# Girder Geometrics
girder_top_flange_area = girder_flange_width * girder_flange_thickness
girder_web_area = girder_web_height * girder_web_thickness
girder_bot_flange_area = girder_flange_width * girder_flange_thickness # //TODO - Sheet is set up so Top Flange=Bottom Flange

In [190]:
# Height values are to centroid
girder_top_flange_centroid_height = girder_flange_thickness + girder_web_height + girder_flange_thickness / 2
girder_web_centroid_height = girder_flange_thickness + girder_web_height / 2
girder_bot_flange_centroid_height = girder_flange_thickness / 2

In [191]:
# Moment of Inertia values
girder_top_flange_I_o = (girder_flange_width * girder_flange_thickness ** 3) / 12
girder_web_I_o = (girder_web_thickness * girder_web_height ** 3) / 12
girder_bot_flange_I_o = (girder_flange_width * girder_flange_thickness ** 3) / 12

In [192]:
# Table Values depend on the following two definitions
girder_height = girder_flange_thickness + girder_web_height + girder_flange_thickness
girder_centroid = girder_height / 2

# Ay^2 Values in table
girder_top_flange_A_y_sq = girder_top_flange_area * (girder_top_flange_centroid_height - girder_centroid) ** 2
girder_web_A_y_sq = girder_web_area * (girder_web_centroid_height - girder_centroid) ** 2
girder_bot_flange_A_y_sq = girder_top_flange_area * (girder_bot_flange_centroid_height - girder_centroid) ** 2

# Out-of-Plane I_o
girder_top_flange_oop_I_o = (girder_flange_thickness * girder_flange_width ** 3) / 12
girder_web_oop_I_o = girder_web_height * girder_web_thickness ** 3 / 12
girder_bot_flange_oop_I_o = (girder_flange_thickness * girder_flange_width ** 3) / 12

In [193]:
# Remaining Girder Section Properties
girder_area = girder_top_flange_area + girder_web_area + girder_bot_flange_area
girder_I_xx = girder_top_flange_I_o + girder_web_I_o + girder_bot_flange_I_o + girder_top_flange_A_y_sq + girder_web_A_y_sq + girder_bot_flange_A_y_sq
girder_S_xx = girder_I_xx / girder_centroid
girder_r_xx = (girder_I_xx / girder_area) ** .5
girder_I_yy = girder_top_flange_oop_I_o + girder_web_oop_I_o + girder_bot_flange_oop_I_o
girder_S_yy = girder_I_yy / (girder_flange_width / 2)
girder_r_yy = (girder_I_yy / girder_area) ** .5
girder_weight_per_ft_no_cont = (girder_area * unit_weight_steel).to('lbf/ft')

# No Holes
girder_S_x_net = girder_S_xx

In [194]:
## Design Summary (No Math, just reprints these values from other sections)
#print(girder_web_height)
#print(girder_web_thickness)
#print(girder_flange_width)
#print(girder_flange_thickness)
#print(L_brace)
#print(trans_stiff_actual_da)
#print(stiffener_width_bst)
#print(stiffener_thickness_tst)
#print(bearing_stiffener_width_bsb)
#print(bearing_stiffener_thickness_tsb)
#print(bearing_stiffener_corner_clip)
#print(bearing_stiff_weld_leg)

## Superstructure DC Calcs

The following values are defined in other sheets,

```python
# Unit Weights
unit_weight_steel = 'Global'!G21
span_length = 'Global'!G14
stop_plate_detail_width = 'Global'!G11

# Girder
girder_area = 'Girder'!F181
girder_length = 'Global'!G15
girder_unit_weight = $E$9 = 'Global'!G21
connections = $E$14
```

Girder calcs depend on the following values, so they were entered here,

In [195]:
# Girder
girder_weight_per_ft = (girder_weight_per_ft_no_cont * (1 + steel_connection_cont)).to('kip/ft')
girder_self_load = (girder_area * girder_length * unit_weight_steel * girder_quantity * (1 + steel_connection_cont)).to('kip')

In [196]:
# Intermediate Floorbeams
int_floorbeam_weight = (girder_spacing * floorbeam.weight * floorbeam_quantity * (1 + steel_connection_cont)).to('kips')

# End Floor Beams
end_floorbeam_weight = (girder_spacing * end_floorbeam.weight * end_floorbeam_quantity * (1 + steel_connection_cont)).to('kips')

total_floorbeam_weight = end_floorbeam_weight + int_floorbeam_weight

In [197]:
# Diaphragms
total_diaphragm_weight = (floorbeam_spacing * diaphragm.weight * diaphragm_quantity * (1 + steel_connection_cont)).to('kips')

In [198]:
# Lateral Bracing # //TODO - lateral bracing quantity seems low
total_lateral_weight = (lateral_bracing_length * lateral_bracing.weight * lateral_bracing_quantity * (1 + steel_connection_cont)).to('kips')

In [199]:
# Floor
deck_plate_lin_weight_per_girder = (deck_plate_thickness * ballast_plates_clear_space * unit_weight_steel * (1 + steel_connection_cont) / 2).to('lbf/ft')
floor_area_load = (deck_plate_thickness * unit_weight_steel * (1 + steel_connection_cont)).to('lbf/ft^2')

total_floor_weight = (deck_plate_thickness * ballast_plates_clear_space * floor_length * unit_weight_steel * floor_quantity * (1+steel_connection_cont)).to('kip')

In [200]:
# Diagonal Stop Plate # //TODO - ask about what this detail is
dia_stop_pl_lin_w_per_girder = (dia_stop_pl_t * dia_stop_pl_width * unit_weight_steel * (1 + steel_connection_cont)).to('lbf/ft')
dia_stop_pl_area_load = (dia_stop_pl_t * dia_stop_pl_width * unit_weight_steel * (1 + steel_connection_cont) / stop_plate_detail_width).to('lbf/ft^2')

dia_stop_pl_weight = (dia_stop_pl_t * dia_stop_pl_width * floorbeam_spacing * unit_weight_steel * dia_stop_pl_quantity * (1 + steel_connection_cont)).to('kip')

In [201]:
# Horizontal Upper Floor Plate # //TODO - Ask about what this detail is
horz_upper_fl_pl_length = 4 * floorbeam_spacing - 8 * units('in') # //TODO - is this always 8"
# //TODO - formula given says (1+C), which is hard coded as 10.4, figure out what this is, also width has bad math in excel
horz_upper_fl_pl_weight_per_girder = (horz_upper_floor_pl_t * horz_upper_floor_pl_width * unit_weight_steel * (1 + steel_connection_cont) * horz_upper_fl_pl_length / (10.4 * units('ft'))).to('lbf/ft')
horz_upper_fl_pl_area_load = (horz_upper_floor_pl_t * horz_upper_floor_pl_width * unit_weight_steel * (1 + steel_connection_cont) / stop_plate_detail_width * horz_upper_fl_pl_length / (10.4 * units('ft'))).to('lbf/ft^2')

horz_upper_fl_pl_weight = (horz_upper_floor_pl_t * horz_upper_floor_pl_width * horz_upper_fl_pl_length * unit_weight_steel * horz_upper_floor_pl_quantity * (1 + steel_connection_cont)).to('kip')

In [202]:
# Waterproofing
waterproofing_width = ballast_plates_clear_space + 2 * dia_stop_pl_width
waterproofing_length = floor_length
waterproofing_unit_weight
waterproofing_weight_per_girder = waterproofing_width * waterproofing_unit_weight / girder_quantity # //TODO - not ref correctly in excel
waterproofing_area_load = waterproofing_unit_weight # //TODO - not used/needless
waterproofing_area_load_2 = waterproofing_unit_weight * (waterproofing_width - 14 * units('ft'))/2/stop_plate_detail_width # //TODO - Values are hardcoded

waterproofing_weight = (waterproofing_width * waterproofing_length * waterproofing_unit_weight * waterproofing_quant).to('kips')

In [203]:
# Asphaltic Plank
asp_plank_width = waterproofing_width
asp_plank_length = floor_length
asphalt_unit_weight
asphaltic_plank_t
asphaltic_plank_quant
asp_lin_weight_per_girder = (asp_plank_width * asphaltic_plank_t * asphalt_unit_weight / girder_quantity).to('lbf/ft') # //TODO - girder quant not ref correctly in excel
asp_plank_area_load = (asphaltic_plank_t * asphalt_unit_weight).to('lbf/ft^2') # Between stop Plates
asp_plank_area_load_2 = (asphaltic_plank_t * asphalt_unit_weight * (asp_plank_width - 14 * units('ft'))/2/stop_plate_detail_width).to('lbf/ft^2') # //TODO - Hardcoded values

asp_plank_weight = (asp_plank_width * asp_plank_length * asphalt_unit_weight * asphaltic_plank_t * asphaltic_plank_quant).to('kips')

In [204]:
# Flooring linear weight per girder
floor_weight_per_girder = asp_lin_weight_per_girder + waterproofing_weight_per_girder + horz_upper_fl_pl_weight_per_girder + dia_stop_pl_lin_w_per_girder + deck_plate_lin_weight_per_girder

# Flooring Load on FBs between stop plates and deck plate
floor_load_on_fbs = (asp_plank_area_load + waterproofing_area_load + floor_area_load).to('lbf/ft^2')

# Flooring Load on FBs over stop plates
floor_load_over_stop_plates = asp_plank_area_load_2 + waterproofing_area_load_2 + horz_upper_fl_pl_area_load + dia_stop_pl_area_load

In [205]:
# Ballast Under Ties
ballast_under_ties_width = (ballast_plates_clear_space + 9/12 * ballast_under_ties_t * 2 /2).to('ft') # //TODO - What is the 9/12?
ballast_under_ties_length = floor_length
ballast_unit_weight
ballast_volume = (ballast_under_ties_width * ballast_under_ties_t * ballast_under_ties_length).to('ft^3')
ballast_area_load = (ballast_under_ties_t * ballast_under_ties_width * ballast_unit_weight / ballast_plates_clear_space).to('lbf/ft^2')
ballast_area_load_2 = (ballast_under_ties_t * ballast_unit_weight).to('lbf/ft^2')

ballast_weight = (ballast_under_ties_t * ballast_under_ties_width * ballast_under_ties_length * ballast_unit_weight * ballast_under_ties_quant).to('kips')

In [206]:
# Ties
tie_depth
tie_width
tie_length
tie_unit_weight
tie_spacing
floor_length
tie_quantity = floor_length / tie_spacing
tie_volume = tie_depth * tie_width * tie_length * tie_quantity
tie_vol_per_ft = (tie_depth * tie_width * tie_length / tie_spacing).to('ft^3/ft')
tie_area_load = (tie_depth * tie_width * tie_length * tie_unit_weight * 1 / tie_spacing / ballast_plates_clear_space).to('lbf / ft^2')
tie_area_load_2 = (tie_depth * tie_width * tie_length * tie_unit_weight * 1 / tie_spacing / tie_length).to('lbf/ft^2')

tie_weight = (tie_depth * tie_width * tie_length * tie_unit_weight * tie_quantity).to('kips')

In [207]:
# Ballast at Tie Level
ballast_tie_lvl_t = tie_depth
ballast_tie_lvl_width = ballast_under_ties_width + 9/12 * ballast_tie_lvl_t
ballast_tie_level_length = ballast_under_ties_length
ballast_unit_weight
tie_volume
ballast_tie_lvl_vol = ((ballast_tie_lvl_t * ballast_tie_lvl_width - tie_level_ballast_sloped_reduction) * ballast_tie_level_length - tie_volume).to('ft^3')
ballast_tie_lvl_area_load = ((ballast_tie_lvl_t * ballast_tie_lvl_width - tie_vol_per_ft - tie_level_ballast_sloped_reduction) * ballast_unit_weight / ballast_plates_clear_space).to('lbf/ft^2')
ballast_tie_lvl_area_load_2 = (ballast_tie_lvl_t * ballast_unit_weight * (1 - (tie_width / tie_spacing))).to('lbf/ft^2')

ballast_tie_lvl_subtotal = (ballast_tie_lvl_t * ballast_tie_lvl_width - tie_level_ballast_sloped_reduction).to('ft^2')

ballast_tie_level_weight = (((ballast_tie_lvl_t * ballast_tie_lvl_width - tie_level_ballast_sloped_reduction) * ballast_tie_level_length - tie_volume) * ballast_unit_weight * tie_level_ballast_quant).to('kip')

In [208]:
# Rail
track_unit_weight
floor_length
rail_quantity
rail_area_load = track_unit_weight / ballast_plates_clear_space
rail_area_load_2 = 25 * units('lbf/ft^2') # (track_unit_weight / tie_length).to('lbf/ft^2') # //TODO - Don't think this is calculating correctly replaced tie width w/ tie length to be closer to reported value

rail_weight = (track_unit_weight * floor_length * rail_quantity).to('kip')

In [209]:
# Future ballast Allowance
future_ballast
future_ballast_width = ballast_plates_clear_space + 9/12 * (ballast_under_ties_t + future_ballast)
ballast_tie_level_length
ballast_unit_weight
future_ballast_quantity
future_ballast_vol = (future_ballast * future_ballast_width * ballast_tie_level_length).to('ft^3')
future_ballast_area_load = (future_ballast * future_ballast_width * ballast_unit_weight / ballast_plates_clear_space).to('lbf/ft^2')
future_ballast_area_load_2 = (future_ballast * ballast_unit_weight).to('lbf/ft^2')

future_ballast_weight = (future_ballast_vol * ballast_unit_weight * future_ballast_quantity).to('kip')

# Summary Calcs
track_weight_per_girder = (ballast_weight + tie_weight + ballast_tie_level_weight + rail_weight + future_ballast_weight) / ballast_tie_level_length
track_area_load_on_fbs = sum([ballast_area_load, tie_area_load, ballast_tie_lvl_area_load, rail_area_load, future_ballast_area_load])
track_assembly_load_on_deck_plate = sum([ballast_area_load_2, tie_area_load_2, ballast_tie_lvl_area_load_2, rail_area_load_2, future_ballast_area_load_2]).to('lbf/ft^2')

In [210]:
# Floorbeam bracket
fb_bracket_flange_t
fb_bracket_flange_web
fb_bracket_flange_len
fb_bracket_web_t
fb_bracket_web_width
fb_bracket_web_height
fb_bracket_quantity
fb_bracket_vol = (fb_bracket_flange_t * fb_bracket_flange_web * fb_bracket_flange_len + fb_bracket_web_t * fb_bracket_web_width * fb_bracket_web_height).to('ft^3')
unit_weight_steel

floorbeam_bracket_weight = ((fb_bracket_vol * unit_weight_steel * fb_bracket_quantity) * (1 + steel_connection_cont)).to('kip')

In [211]:
list_of_dead_load_factors = [
    floorbeam_bracket_weight,
    future_ballast_weight,
    rail_weight,
    ballast_tie_level_weight,
    tie_weight,
    ballast_weight,
    asp_plank_weight,
    waterproofing_weight,
    horz_upper_fl_pl_weight,
    dia_stop_pl_weight,
    total_floor_weight,
    total_lateral_weight,
    total_diaphragm_weight,
    total_floorbeam_weight,
    girder_self_load
]   

In [212]:
lifting_dead_load_factors = [
    floorbeam_bracket_weight,
    rail_weight,
    tie_weight,
    ballast_weight,  # //TODO - Shouldn't this not be included here?
    asp_plank_weight,
    waterproofing_weight,
    horz_upper_fl_pl_weight,
    dia_stop_pl_weight,
    total_floor_weight,
    total_lateral_weight,
    total_diaphragm_weight,
    total_floorbeam_weight,
    girder_self_load
]

In [213]:
total_dead_load = sum(list_of_dead_load_factors)

In [214]:
total_dead_load_lifting = sum(lifting_dead_load_factors)

In [215]:
distributed_load = total_dead_load / span_length / girder_quantity  # //TODO - Girders hardcoded in excel sheet
distributed_load

4.450551053178248 <Unit('kip / foot')>

## Remaining Girder Calcs/Checks

In [216]:
floorbeam_load_on_girder = total_floorbeam_weight / girder_quantity / span_length
diaphragm_load_on_girder = total_diaphragm_weight / girder_quantity / span_length

In [217]:
# Floor System
floor_weight_per_girder
total_lateral_weight
girder_quantity
span_length

lat_bracing_load_on_girder = total_lateral_weight / girder_quantity / span_length

In [218]:
track_assembly = track_weight_per_girder
total_dl_over_total_length = sum([girder_weight_per_ft, floorbeam_load_on_girder, diaphragm_load_on_girder, floor_weight_per_girder, lat_bracing_load_on_girder, track_assembly])

w = total_dl_over_total_length
L = span_length

M_dl = (w * L ** 2 / 8).to('kips*ft')

In [219]:
span_live_load = (E80_55_ft - E80_50_ft) / (55 * units.ft - 50 * units.ft) * (span_length - 50 * units.ft) + E80_50_ft # //TODO - Values are hardcoded
M_ll = span_live_load

In [220]:
# Impact Load      AREMA 15.1.3.5.d
impact_percent = (40 - 3 * span_length ** 2 / (1600 * units('ft^2'))) / 100 # //TODO - Hardcoded Values, make sure they match AREMA Formula

M_i = (M_ll * ballasted_deck_reduction * impact_percent) # //TODO - Dimensionality off, /100 should adjustment probably be in the percentage calc

In [221]:
# Rocking Effect    AREMA 15.1.3.5.d
eqiv_uniform_live_load = 8 * M_ll / span_length ** 2

P = eqiv_uniform_live_load * rocking_percent

a = (girder_spacing-railroad_gage)/2

load_on_girder_rocking = P * (1 - 2 * a / girder_spacing)

M_re = load_on_girder_rocking * L ** 2 / 8

In [222]:
# Centrifugal Force    AREMA 15-1.3.6
design_speed = 35 * units('miles/hour')      # //TODO - Hardcoded Values, should be inputs
curve_radius = 2000 * units('ft')            # //TODO - Hardcoded Values, should be inputs
deg_curve = 2*math.atan(100/(2*curve_radius.magnitude)) * units('rad')  # //TODO - Find a better way to handle units
centrifugal_percentage = (0.00117 * design_speed ** (2 * deg_curve))

In [223]:
# Longitudinal Force not considered

In [224]:
# Wind Load    # AREMA 15-1.3.7
wind_load_8_ft_offset = 300 * units('lbf/ft')
P = 8 * units('ft') * wind_load_8_ft_offset / railroad_gage
a = (girder_spacing-railroad_gage) / 2

girder_reaction_from_wind = (P * (1 - 2 * a / girder_spacing)).to('kip/ft')

V_wind = girder_reaction_from_wind * span_length / 2
M_wind = girder_reaction_from_wind * span_length ** 2 / 8

In [225]:
# Total Moment
M_dl
M_ll
M_i
M_re
M_wind

# Case 1 = M_dl + M_ll + M_i + M_re
case_1 = sum([M_dl, M_ll, M_i, M_re])

# Case 2 = (M_dl + M_ll + M_i + M_wind) / 1.25
case_2 = sum([M_dl, M_ll, M_i, M_wind]) / 1.25

M_tot = max(case_1, case_2)

In [226]:
# Req. Section Modulus
F_b = (0.55 * F_y).to('kips/in^2')
S_req_moment = (M_tot / F_b).to('in^3')

In [227]:
# Fatigue Section Required
impact_percent          # AREMA 15 - Table 15-1-7, Table 15-1-8
M_if = M_i * impact_percent
M_f = M_ll + M_if

F_b_fat = 16 * units('ksi') # AREMA 15 - Table 15-1-10

S_req_fat = (M_f / F_b_fat).to('in^3')

In [228]:
# Required Section Modulus
S_req = max(S_req_moment, S_req_fat)

In [229]:
# Deflection requirements for member sizing
delta_max = (span_length / 640).to('inch')      # AREMA 15-1.2.5b

M_ll
M_i
M_ll_and_M_i = M_ll + M_i

w_deflection = (M_ll_and_M_i * 8 / span_length ** 2).to('kip/in')

I_x_req = (5 * w_deflection * span_length ** 4 / (384 * E_steel * delta_max)).to('in^4')

In [230]:
# Compression Flange Check
(L_brace / girder_r_yy).to('in/in')
E / F_y

5.55 * (E / F_y) ** .5

# Flange Compression Check
if (L_brace / girder_r_yy) <= 5.55 * (E / F_y) ** .5:
    print(colored('OK', 'green'))
else:
    print(colored('No Good - Compression Flange Check Failed', 'red'))

OK


In [231]:
# Web Thickness     AREMA 15-1.7.3
girder_web_thickness
girder_web_height
F_y / E
0.18 * (F_y / E) ** .5 * girder_web_height

# Web Thickness Check 1
if girder_web_thickness >= 0.18 * (F_y / E) ** .5 * girder_web_height:
    print(colored('OK', 'green'))
else:
    print(colored('No Good - Web Compression Check Failed', 'red'))

girder_flange_thickness / 6

# Web Thickness Check 2
if girder_web_thickness >= girder_flange_thickness / 6:
    print(colored('OK', 'green'))
else:
    print(colored('No Good - Web Compression Check Failed', 'red'))

OK
OK


In [232]:
# Outstanding Elements in Compression (AREMA 15-1.6.2)
girder_flange_width / 2

0.43 * girder_flange_thickness * (E / F_y) ** .5

girder_flange_thickness # //TODO - Not used in excel sheet

# Flange_compression
if girder_flange_width / 2 <= 0.43 * girder_flange_thickness * (E / F_y) ** .5:
    print(colored('OK', 'green'))
else:
    print(colored('No Good - Compression Flange Check Failed', 'red'))

OK


In [233]:
# Allowable Stress
F_b_ten = 0.55 * F_y # AREMA Table 15-1-12
comp_value_1 = 0.55 * F_y - ((0.55 * F_y ** 2) / (6.3 * math.pi ** 2 * E_steel)) * (L_brace / girder_r_yy) ** 2

girder_depth = girder_web_height + 2 * girder_flange_thickness
girder_flange_area = girder_flange_width * girder_flange_thickness

comp_value_2 = ((0.131 * math.pi * E_steel) / ((L_brace * girder_depth * (1 + poisson_ratio) ** .5) / girder_flange_area)).to('psi')

comp_value_3 = 0.55 * F_y

F_b_comp = min(comp_value_3, max(comp_value_2, comp_value_1))

In [234]:
# Stress in Extreme fibers of Girder
M_tot
girder_S_xx
F_b_comp_act = (M_tot / girder_S_xx).to('lbf/in^2')

# Bending Check
if F_b_comp_act <= F_b_comp:
    print(colored(f'OK, Stress Ratio: {F_b_comp_act/F_b_comp}', 'green'))
else:
    print(colored('No Good - Girder Bending Check Failed', 'red'))

OK, Stress Ratio: 0.9225909060826183 force_pound / inch ** 2 / pound_force_per_square_inch


In [235]:
F_b_ten_act = (M_tot / girder_S_xx).to('psi')
F_b_ten

# Bending Check
if F_b_ten_act <= F_b_ten:
    print(colored(f'OK, Stress Ratio: {F_b_ten_act/F_b_ten}', 'green'))
else:
    print(colored('No Good - Girder Bending Check Failed', 'red'))

OK, Stress Ratio: 0.9225909060826186 dimensionless


In [57]:
# Fatigue (AREMA 15-1.3.13)
M_f
girder_S_xx
S_r_fat_actual = (M_f / girder_S_xx).to('psi')

S_r_fat = F_b_fat.to('psi')

# Bending Fatigue
if S_r_fat_actual <= S_r_fat:
    print(colored(f'OK, Stress Ratio: {S_r_fat_actual/S_r_fat}', 'green'))
else:
    print(colored('No Good - Girder Bending Fatigue Check Failed', 'red'))

OK, Stress Ratio: 0.73380327114963 dimensionless


In [58]:
F_b_fat

16 <Unit('kip_per_square_inch')>

In [59]:
# Deflection AREMA 15-1.2.5
delta_max
M_ll
M_i

w = ((M_ll + M_i) / (span_length ** 2) * 8)
E_steel
girder_I_xx

delta_total = ((5 * w * (span_length ** 4)) / (384 * E_steel * girder_I_xx)).to('in')

# Deflection # //TODO - Not sure the ratio matters as much for this one
if delta_total <= delta_max:
    print(colored(f'OK, Ratio: {delta_total/delta_max}', 'green'))
else:
    print(colored('No Good - Deflection Check Failed', 'red'))

OK, Ratio: 0.6307157084746534 dimensionless


In [60]:
# Dead Load Camber
w_dl = total_dl_over_total_length.to('kip/in')
delta_dl = (delta_total * w_dl / w).to('in')

In [61]:
# Web Shear
V_dl = (w_dl * span_length / 2).to('kip')

# Shear from AREMA 15 Table 1-16
shear_50_ft = 174.4 * units('kips')
shear_55_ft = 185.31 * units('kips')
V_ll = (shear_55_ft - shear_50_ft) / (55 * units('ft') - 50 * units('ft')) * (span_length - 50 * units('ft')) + shear_50_ft

impact_percent

V_imp = 0.9 * V_ll * impact_percent     # AREMA 15-1.3.5.b

R1_rocking_force = load_on_girder_rocking
V_re = R1_rocking_force * L / 2
V_wind

V_tot = V_dl + V_ll + V_re + V_imp

F_r = V_tot / (girder_web_height * girder_web_thickness)

F_v = (0.35 * F_y).to('kips/in^2')

# Web Shear AREMA Table 15-1-12 check # //TODO - This Check isn't comparing like units in the excel sheet?, think it should be as follows,
if F_r <= F_v:
    print(colored(f'OK, Stress Ratio: {F_r/F_v}', 'green'))
else:
    print(colored('No Good - Web Shear Check Failed', 'red'))

OK, Stress Ratio: 0.6164890919227507 dimensionless


In [62]:
# Allowable Stress on welds
F_v
V_tot
Q = girder_flange_width * girder_flange_thickness * (girder_web_height / 2 + girder_flange_thickness / 2)
F_r = V_tot * Q / (girder_I_xx * girder_web_thickness)

# Weld Strength
if F_r <= F_v:
    print(colored(f'OK, Stress Ratio: {F_r/F_v}', 'green'))
else:
    print(colored('No Good - Weld Strength Check Failed', 'red'))

OK, Stress Ratio: 0.5075830449480359 dimensionless


In [63]:
# Web plate stiffeners
girder_web_height
trans_stiff_check = (2.12 * (E_steel / F_y) ** .5 * girder_web_thickness) # AREMA 15-1.7.8.a

# Transverse stiffener check
if trans_stiff_check <= girder_web_height:
    print(colored('Transverse Stiffeners Required', 'red'))
else:
    print(colored('Transverse Stiffeners NOT Required', 'green'))


# //TODO - Look into this value, it's not used 
clear_dist_to_prev_web_shear_buck = 1.95 * (E_steel * girder_S_xx) ** .5

Q = girder_flange_width * girder_flange_thickness * (girder_web_height + girder_flange_thickness) / 2 + girder_web_height * girder_web_thickness / 2 * girder_web_height / 4

V_tot
girder_I_xx
girder_web_thickness
S = f_v = V_tot * Q / (girder_I_xx * girder_web_thickness)

d = 1.95 * girder_web_thickness * (E_steel / S) ** .5

max_clear_dist = min([d, trans_stiff_da, girder_web_height])

# Weld Strength
if trans_stiff_actual_da <= max_clear_dist or girder_web_height <= trans_stiff_check:
    print(colored('OK', 'green'))
else:
    print(colored('No Good - Stiffener Spacing Check Failed', 'red'))

D_d = max(1, min(5, girder_web_height / d))

# Required Moment of inertia (Stiffener) AREMA 15-1.7.8.a
stiff_I_xx_req = 2.5 * trans_stiff_actual_da * girder_web_thickness ** 3 * (D_d ** 2 - 0.7)

stiff_I_xx_act = stiffener_thickness_tst * stiffener_width_bst ** 3 / 12 + stiffener_thickness_tst * stiffener_width_bst * (girder_web_thickness / 2 + stiffener_width_bst / 2) ** 2

# Stiffener I_xx Check
if stiff_I_xx_act >= stiff_I_xx_req or girder_web_height <= trans_stiff_check:
    print(colored('OK', 'green'))
else:
    print(colored('No Good - Stiffener Spacing Check Failed', 'red'))

# Max Stiffener Width AREMA 15-1.7.8.b
max_stiff_width = 16 * stiffener_thickness_tst
min_stiff_width = 2 * units('in') + girder_height / (30)

stiff_t_check_1 = stiffener_width_bst <= max_stiff_width and stiffener_width_bst >= min_stiff_width
stiff_t_check_2 = girder_web_height <= trans_stiff_check

# Stiffener Thickness Check
if stiff_t_check_1 or stiff_t_check_2:
    print(colored('OK', 'green'))
else:
    print(colored('No Good - Stiffener Thickness Check Failed', 'red'))

Transverse Stiffeners Required
OK
OK
OK


In [64]:
# Transverse Stiffener Weld Fatigue Stress
M_f
d = girder_web_height / 2
S_stiff = girder_I_xx / d
sr_weld = (M_f / S_stiff).to('psi')

F_b_fat_trans = 12000 * units('psi') # Stress category C' N>2,000,000 cycles AREMA 15-1-9

# Stiffener Weld Fatigue
if sr_weld <= F_b_fat_trans:
    print(colored(f'OK, Stress Ratio: {sr_weld/F_b_fat_trans}', 'green'))
else:
    print(colored('No Good - Stiffener Weld Fatigue Check Failed', 'red'))

OK, Stress Ratio: 0.9281306196358959 dimensionless


In [65]:
# Longitudinal Plate Stiffeners
girder_web_height
4.18 * (E_steel / F_y) ** .5 * girder_web_thickness # AREMA 15-1.7.8.f

# Longitudinal Stiffeners
if girder_web_height <= 4.18 * (E_steel / F_y) ** .5 * girder_web_thickness:
    long_stiff_check = 'Not Required'
    print(colored('Longitudinal Stiffeners not required', 'green'))
else:
    print(colored('Longitudinal Stiffeners Required', 'red'))

# Longitudinal Stiffeners   # //TODO - Bad Check
if long_stiff_check == 'Not Required':
    print(colored('OK', 'green'))
else:
    print(colored('No Good - Longitudinal Stiffeners Check', 'red'))

Longitudinal Stiffeners not required
OK


In [66]:
# Bearing Stiffener
bearing_stiff_limiting_ratio = (0.43 * bearing_stiffener_thickness_tsb * ((E_steel / F_y) ** 0.5)).to('in')

# Longitudinal Stiffeners   # //TODO - Bad Check
if bearing_stiffener_width_bsb <= bearing_stiff_limiting_ratio:
    print(colored('OK', 'green'))
else:
    print(colored('No Good - Bearing Stiffeners w/t ratio Check', 'red'))
    
# Outstanding element in compression check
girder_web_thickness
effective_web_length = 25 * girder_web_thickness   # AREMA 15-1.7.7.c

bearing_stiff_area = girder_web_thickness * effective_web_length + 2 * bearing_stiffener_width_bsb * bearing_stiffener_thickness_tsb
I_xx_web = 2 * (bearing_stiffener_width_bsb ** 3 * bearing_stiffener_thickness_tsb / 12 + bearing_stiffener_width_bsb * bearing_stiffener_thickness_tsb * (girder_web_thickness/2 + bearing_stiffener_width_bsb/2)**2) + girder_web_thickness**3 * effective_web_length/12
I_xx_stiff = 2*bearing_stiffener_thickness_tsb**3*bearing_stiffener_width_bsb/12+effective_web_length**3*girder_web_thickness/12

controlling_moment_of_inertia = min(I_xx_web, I_xx_stiff)

# AREMA 15-1.7.7-c
effective_length = 0.75 * girder_web_height
bear_stiff_r_y = (controlling_moment_of_inertia / bearing_stiff_area) ** 0.5

bear_stiff_slenderness_ratio = effective_length / bear_stiff_r_y

(F_y / E_steel) ** .5

0.629 / (F_y / E) ** .5
5.034 / (F_y / E) ** .5

# AREMA Table 15-1-12
case_1_all_stress = 0.55 * F_y
case_2_all_stress = (0.6 * F_y.magnitude - (17500 * F_y / E_steel) ** (3/2) * bear_stiff_slenderness_ratio) * units('psi')
case_3_all_stress = 0.514*math.pi**2*E_steel/bear_stiff_slenderness_ratio**2

# Controlling Case
if bear_stiff_slenderness_ratio <= 0.629 / (F_y / E) ** .5:
    print(colored('Case 1 Controls', 'green'))
    print(bearing_stiff_F_a := case_1_all_stress)
elif bear_stiff_slenderness_ratio <= 5.034 / (F_y / E) ** .5:
    print(colored('Case 2 Controls', 'green'))
    print(bearing_stiff_F_a := case_2_all_stress)
else:
    print(colored('Case 3 Controls', 'green'))
    print(bearing_stiff_F_a := case_3_all_stress)

OK
Case 2 Controls
27155.276272613875 pound_force_per_square_inch


In [67]:
V_tot

bearing_stress_act = (V_tot / bearing_stiff_area).to('psi')

# Longitudinal Stiffeners
if bearing_stress_act <= bearing_stiff_F_a:
    print(colored(f'OK, Stress Ratio: {bearing_stress_act/bearing_stiff_F_a}', 'green'))
else:
    print(colored('No Good - End Bearing Compression Check', 'red'))

OK, Stress Ratio: 0.5112597758960217 dimensionless


In [68]:
# Bearing Stiff Weld      AREMA 15-1.7.7.a
fillet_weld_area = 4 * 0.707 * bearing_stiffener_fillet_weld_leg * (girder_web_height - 2 * bearing_stiffener_corner_clip)
V_tot

weld_stress_act = (V_tot / fillet_weld_area).to('psi')

allowable_weld_stress = min(19000 * units('psi'), 0.35*F_y) # AREMA Table 15-1-14

# Bearing Stiff Weld Check
if weld_stress_act <= allowable_weld_stress:
    print(colored(f'OK, Stress Ratio: {weld_stress_act/allowable_weld_stress}', 'green'))
else:
    print(colored('No Good - Bearing Stiff Weld Check', 'red'))

OK, Stress Ratio: 0.45102357600552445 dimensionless


In [69]:
# Bearing Stiff bearing area
stiff_bearing_area = 2 * bearing_stiffener_thickness_tsb * (bearing_stiffener_width_bsb - 2 * bearing_stiffener_corner_clip)

In [70]:
stiff_bearing_area
V_tot
bearing_area_stress_act = (V_tot / stiff_bearing_area).to('psi')
bearing_area_stress_all = (.83 * F_y).to('psi')

# Bearing area allowable stress
if bearing_area_stress_act <= bearing_area_stress_all:
    print(colored(f'OK, Stress Ratio: {bearing_area_stress_act/bearing_area_stress_all}', 'green'))
else:
    print(colored('No Good - Bearing Stiff Weld Check', 'red'))

OK, Stress Ratio: 0.6340616578694178 dimensionless


## Floor Beam Calcs

In [71]:
# Dead Load
# Floor Beams
floorbeam.weight

# Diaphragms
diaphragm.weight = 61 * units('lbf/ft') # //TODO - Doesn't match section: W16x89
diaphragm_quantity = 1 # //TODO - Why is this only 1 in excel?

diaphragm_length = floorbeam_spacing

lateral_bracing_length = 8.20 * units('ft') # //TODO - why is this redefined and lower

diaphragm_P = diaphragm.weight*diaphragm_quantity*diaphragm_length
bracing_P = lateral_bracing.weight * bracing_quant * lateral_bracing_length

floorbeam_dl_V = (floorbeam.weight * girder_spacing) / 2
floorbeam_dl_M = floorbeam.weight * girder_spacing ** 2 / 8

diaphragm_dl_V = diaphragm_P / 2
diaphragm_dl_M = diaphragm_P * girder_spacing / 4

lateral_bracing_V = bracing_P / 2
lateral_bracing_M = bracing_P * girder_spacing / 4

In [72]:
# Floor assembly between stop plates
floor_load_on_fbs
ballast_plates_clear_space

floor_assembly_V = ballast_plates_clear_space * floor_load_on_fbs * floorbeam_spacing / 2
floor_assembly_M = (floor_load_on_fbs * floorbeam_spacing * ballast_plates_clear_space) / 4 * (girder_spacing - ballast_plates_clear_space / 2) 

In [73]:
# Floor assembly over stop plates
floor_load_over_stop_plates
fb_applied_width = (girder_spacing - ballast_plates_clear_space) / 2

floor_over_stop_pl_V = floor_load_over_stop_plates * fb_applied_width * floorbeam_spacing
floor_over_stop_pl_M = floor_load_over_stop_plates * fb_applied_width ** 2 / 2 * floorbeam_spacing

In [74]:
# Track assembly weight
track_area_load_on_fbs
ballast_plates_clear_space

track_area_load_V = track_area_load_on_fbs*ballast_plates_clear_space/2*floorbeam_spacing
track_area_load_M = track_area_load_on_fbs * ballast_plates_clear_space / 4 * (girder_spacing - ballast_plates_clear_space / 2) * floorbeam_spacing

In [75]:
total_fb_dl_shear = sum([
    floorbeam_dl_V,
    diaphragm_dl_V,
    lateral_bracing_V,
    floor_assembly_V,
    floor_over_stop_pl_V,
    track_area_load_V
])

In [76]:
total_fb_dl_moment = sum([
    floorbeam_dl_M,
    diaphragm_dl_M,
    lateral_bracing_M,
    floor_assembly_M,
    floor_over_stop_pl_M,
    track_area_load_M
])

In [77]:
# Live Load
# AREMA 15-1.3.4.2.3 Alternative live load axel
fb_P_ll = 1.15*fb_A*floorbeam_spacing/fb_S
fb_P_2_ll = fb_P_ll / 2

fb_V_ll = fb_P_2_ll
fb_M_ll = fb_P_ll * fb_a / 2

In [78]:
# Impact Live Load
fb_impact_ll_percent = (40 - 3 * girder_spacing.magnitude ** 2 / 1600)/100
fb_P_2_ll

fb_P_imp_ll = fb_impact_ll_percent * fb_P_2_ll * ballasted_deck_reduction

fb_M_imp = fb_P_imp_ll * fb_a

In [79]:
# Rocking Load  AREMA 1.3.5.d
rocking_percent
fb_P_2_ll
R1_rocking = fb_P_2_ll * rocking_percent * (1 - 2 * a / girder_spacing)

fb_M_re = R1_rocking * a

In [80]:
# Wind Force on Loaded Bridge
wind_load = 300 * units('lbf/ft')
fb_pws = wind_load*h*floorbeam_spacing/railroad_gage

fb_wind_R1 = fb_pws * (1-2*a/girder_spacing)

M_wl = fb_wind_R1 * fb_a

In [81]:
# Required Section Properties  # AREMA Table 15-1-11
fb_M_tot_1 = sum([total_fb_dl_moment, fb_M_ll, fb_M_imp, M_re])
fb_M_tot_2 = sum([total_fb_dl_moment, fb_M_ll, fb_M_imp, M_re, M_wl])/1.25
fb_M_tot = max(fb_M_tot_1, fb_M_tot_2)

fb_S_x_req = (fb_M_tot / (0.55 * F_y)).to('in^3')

In [82]:
# Holes - Assume 4 x 1" holes in web for diaphragm
D1 = 3 * units('in') # //TODO - is this a constant?
D2 = 3 * units('in') + D1

I_holes_web = (diaphragm_num_holes/2*(diaphragm_dia_hole*floorbeam.web_thickness)*(D1)**2)+(diaphragm_num_holes/2*(diaphragm_dia_hole*floorbeam.web_thickness)*(D2)**2)

In [83]:
I_holes_flange = lateral_num_holes * lateral_dia_holes * floorbeam.flange_thickness * (floorbeam.depth / 2 - floorbeam.flange_thickness / 2) ** 2

In [84]:
I_net = floorbeam.I_x - I_holes_web - I_holes_flange

In [85]:
I_net

S_x_net = I_net / (floorbeam.depth / 2)

In [86]:
# Allowable Stress (AREMA Table 15-1-12)
Fb_ten = 0.55 * F_y

# Compression in extreme fiber
Fb_comp_1 = 0.55*F_y-(0.55*F_y**2/(6.3*math.pi**2*E_steel))*(diaphragm_spacing_max/floorbeam.r_y)**2
Fb_comp_2 = (0.131 * math.pi * E_steel / (diaphragm_spacing_max * floorbeam.depth * (1+poisson_ratio) ** .5 / (floorbeam.flange_thickness * floorbeam.flange_width))).to('psi')
Fb_comp_3 = 0.55 * F_y

F_b_comp = (min(Fb_comp_3, max(Fb_comp_1, Fb_comp_2))).to('ksi')

f_b_act = (fb_M_tot / S_x_net).to('ksi')

# Bearing area allowable stress
if f_b_act < F_b_comp:
    print(colored(f'OK, Stress Ratio: {f_b_act/F_b_comp}', 'green'))
else:
    print(colored('No Good - Floor Beam Allowable Stress Check', 'red'))

OK, Stress Ratio: 0.3412387259725499 dimensionless


In [87]:
# End Shear
R_dl = total_fb_dl_shear
R_ll = fb_P_ll
R_imp = fb_P_imp_ll * 2
R_re = R1_rocking
R_wl = fb_wind_R1

R_tot = sum([R_dl, R_ll, R_imp, R_re, R_wl])

# Assume 6x1" dia holes in web to connect to girder
fs_net = (R_tot/((floorbeam.depth-web_conn_girder_holes_num*web_conn_girder_holes_dia)*floorbeam.web_thickness)).to('ksi')

F_v = (0.35 * F_y).to('ksi')

# Bearing area allowable stress
if fs_net < F_v:
    print(colored(f'OK, Stress Ratio: {fs_net/F_v}', 'green'))
else:
    print(colored('No Good - Floor Beam End Shear Check', 'red'))

OK, Stress Ratio: 0.2502732273083372 dimensionless


In [88]:
# Bolted Connection - Checked in sep. calc, they account for combined efect so no 1.25 increase
# AREMA 15 1.5.9.a.1 # //TODO - Investigate this standard
R_tot

92465.94656133825 <Unit('force_pound')>

In [89]:
# Floor beam fatigue
fb_M_ll
fb_fat_imp_M = assumed_mean_impact_perc * fb_M_imp
fb_M_fat = fb_M_ll + fb_fat_imp_M

fb_live_load_stress_range = (fb_M_fat / S_x_net).to('ksi')

# N >= 2,000,000 Category B Table 15-1-10 Detail 2.2
fb_fsr = 16 * units('ksi')

# Bearing area allowable stress
if fb_live_load_stress_range < fb_fsr:
    print(colored(f'OK, Stress Ratio: {fb_live_load_stress_range/fb_fsr}', 'green'))
else:
    print(colored('No Good - Floor Beam Fatigue Check', 'red'))

OK, Stress Ratio: 0.32341725842958674 dimensionless


In [90]:
# Deflection
girder_spacing
floorbeam.I_x
E_steel

fb_defl_M = fb_M_ll + fb_M_imp + fb_M_re

fb_w = 8 * fb_defl_M / (girder_spacing ** 2)

fb_deflection = ((5 * fb_w * girder_spacing ** 4) / (384 * E_steel * floorbeam.I_x)).to('in')
max_deflection = (girder_spacing / 640).to('in')

# Deflection Check
if fb_deflection <= max_deflection:
    print(colored(f'OK, Stress Ratio: {fb_deflection/max_deflection}', 'green'))
else:
    print(colored('No Good - Floor Beam Fatigue Check', 'red'))

OK, Stress Ratio: 0.24964506342823886 dimensionless


## Deck Plate Calcs

In [91]:
# Key Dimensions
track_distribution_width = tie_design_width+min_ballast_below_tie # AREMA 1.3.4.2.2.b
track_dist_length = 3 * units('ft') + min_ballast_below_tie # AREMA 1.3.4.2.2.a

In [92]:
# Dead Loads
flooring_load = floor_load_on_fbs
track_load = track_assembly_load_on_deck_plate

In [93]:
# Live Load
axel_load
longitudinal_dist = track_dist_length
lateral_dist = track_distribution_width

dist_axel_load = axel_load / (longitudinal_dist*lateral_dist)

In [94]:
# Impact Load # AREMA 15.1.3.5
L = floorbeam_spacing
deck_pl_impact_L = (40 - (3*L**2 / (1600 * units('ft^2')))) * .9

impact_load = (dist_axel_load * deck_pl_impact_L / 100).to('lbf/ft^2')
impact_load

1209.7005882352948 <Unit('force_pound / foot ** 2')>

In [95]:
# Rocking Effect
rocking_percent

rocking_effect = rocking_percent * dist_axel_load / 2

In [96]:
deck_pl_dead_load = flooring_load + track_load
deck_pl_live_load_impact_re = dist_axel_load + impact_load + rocking_effect

total_dist_load = deck_pl_dead_load + deck_pl_live_load_impact_re

In [97]:
# Compute Forces and Stresses over a 1' unit width
flange_width = min(end_floorbeam.flange_width, floorbeam.flange_width)

floorbeam_spacing
flange_width.to('ft')

M1_w = (1/12*((3*floorbeam_spacing*flange_width/2)-(floorbeam_spacing**2)-(3*flange_width**2/8))).to('ft^2')
M2_w = floorbeam_spacing ** 2 / 24

gov_M_w = max(abs(M1_w), abs(M2_w))

deck_pl_dl_M = gov_M_w * deck_pl_dead_load
deck_pl_ll_M = gov_M_w * deck_pl_live_load_impact_re
deck_pl_tot_M = deck_pl_dl_M + deck_pl_ll_M

deck_pl_S_x = deck_plate_width * deck_plate_thickness ** 2 / 6

# //TODO - Check Units on this one
deck_pl_F_b = (deck_pl_tot_M * (12*units('in')) / deck_pl_S_x).to('psi')

deck_pl_F_b_allow = (0.55 * F_y)

# Stress Check
if deck_pl_F_b <= deck_pl_F_b_allow:
    print(colored(f'OK, Stress Ratio: {deck_pl_F_b/deck_pl_F_b_allow}', 'green'))
else:
    print(colored('No Good - Floor Plate Allowable Stress Check', 'red'))

OK, Stress Ratio: 0.5755581290899663 dimensionless


In [98]:
# Floor Plate Deflection
deck_plate_width
deck_plate_thickness
deck_plate_I_x = deck_plate_width * deck_plate_thickness ** 3 / 12
deck_plate_span_L = floorbeam_spacing
deck_pl_live_load_impact_re

deck_pl_live_load_deflection = (deck_pl_live_load_impact_re*units('in')*floorbeam_spacing**4)/(384*E_steel*deck_plate_I_x)
deck_pl_allow_deflection = (floorbeam_spacing / 640).to('in').to('in')

deck_pl_live_load_deflection = deck_pl_live_load_deflection.to('in')

# Deflection Check
if deck_pl_live_load_deflection <= deck_pl_allow_deflection:
    print(colored(f'OK, Stress Ratio: {deck_pl_live_load_deflection/deck_pl_allow_deflection}', 'green'))
else:
    print(colored('No Good - Floor Plate Deflection Check', 'red'))

OK, Stress Ratio: 0.14099386081451015 dimensionless


## End Floor Beam Calcs

In [99]:
# Floor Beams
end_floorbeam.weight
jack_pt_offset
# end_bearing_stiff_width = (end_floorbeam.flange_width - end_floorbeam.web_thickness) / 2 * 4 / 4
end_bearing_stiff_width = 5.75 * units('in')
bearing_stiffener_t
bearing_stiffener_corner_clip
bearing_stiff_weld_leg

# Geometrics
end_fb_spacing = 2.6 * units('ft') # //TODO - References VOID fb - non-bracing sheet
flooring_on_girder = end_fb_spacing/2+(floor_length-span_length)/2
girder_spacing
ballast_plates_clear_space
diaphragm_spacing_max
railroad_gage
F_y
F_u
E_steel
unit_weight_steel
railroad_gage # //TODO - Defined twice in excel

4.708333333333333 <Unit('foot')>

In [100]:
# Dead Load
# Diaphragms
diaphragm.weight = 61 * units('lbf/ft') # //TODO - Doesn't match section: W16x89
diaphragm_quantity = 1 # //TODO - Why is this only 1 in excel?

diaphragm_length = floorbeam_spacing / 2

lateral_bracing_length = 8.20 * units('ft') # //TODO - why is this redefined and lower


diaphragm_P = diaphragm.weight*diaphragm_quantity*diaphragm_length


end_floorbeam_dl_V = (end_floorbeam.weight * girder_spacing) / 2
end_floorbeam_dl_M = end_floorbeam.weight * girder_spacing ** 2 / 8

end_diaphragm_dl_V = diaphragm_P / 2
end_diaphragm_dl_M = diaphragm_P * end_bearing_stiff_width / 4 # //TODO - Verify End stiff width

In [101]:
# //TODO - Verify no bracing on end floorbeams
# bracing_P = lateral_bracing.weight * bracing_quant * lateral_bracing_length
# end_lateral_bracing_V = bracing_P / 2
# end_lateral_bracing_M = bracing_P * girder_spacing / 4

In [102]:
# Floor assembly between stop plates
floor_load_on_fbs
ballast_plates_clear_space

end_floor_assembly_V = ballast_plates_clear_space * floor_load_on_fbs * flooring_on_girder / 2 # //TODO - Verify Flooring on girder, not fb spacing
end_floor_assembly_M = (floor_load_on_fbs * flooring_on_girder * ballast_plates_clear_space) / 4 * (girder_spacing - ballast_plates_clear_space / 2) 

In [103]:
# Floor assembly over stop plates
floor_load_over_stop_plates
end_fb_applied_width = (girder_spacing - ballast_plates_clear_space) / 2

end_floor_over_stop_pl_V = floor_load_over_stop_plates * end_fb_applied_width * flooring_on_girder
end_floor_over_stop_pl_M = floor_load_over_stop_plates * end_fb_applied_width ** 2 / 2 * flooring_on_girder

In [104]:
# Track assembly weight
track_area_load_on_fbs
ballast_plates_clear_space

end_track_area_load_V = track_area_load_on_fbs*ballast_plates_clear_space/2*flooring_on_girder
end_track_area_load_M = track_area_load_on_fbs * ballast_plates_clear_space / 4 * (girder_spacing - ballast_plates_clear_space / 2) * flooring_on_girder

In [105]:
total_end_fb_dl_shear = sum([
    end_floorbeam_dl_V,
    end_diaphragm_dl_V,
    end_floor_assembly_V,
    end_floor_over_stop_pl_V,
    end_track_area_load_V
])

In [106]:
total_end_fb_dl_moment = sum([
    end_floorbeam_dl_M,
    end_diaphragm_dl_M,
    end_floor_assembly_M,
    end_floor_over_stop_pl_M,
    end_track_area_load_M
])

In [107]:
# Live Load
# AREMA 15-1.3.4.2.3 Alternative live load axel
end_fb_P_ll = 1.15*fb_A*floorbeam_spacing/fb_S
end_fb_P_2_ll = fb_P_ll / 2

end_fb_A = fb_A
end_fb_D = floorbeam_spacing
end_fb_S = fb_S
end_fb_a = fb_a

end_fb_M_ll = end_fb_P_ll * end_fb_a / 2

In [108]:
# Impact Live Load
end_fb_impact_ll_percent = (40 - 3 * girder_spacing.magnitude ** 2 / 1600)/100
end_fb_P_2_ll

end_fb_P_imp_ll = end_fb_impact_ll_percent * end_fb_P_2_ll * ballasted_deck_reduction

end_fb_M_imp = fb_P_imp_ll * fb_a

In [109]:
# Rocking Load  AREMA 1.3.5.d
rocking_percent
end_fb_P_2_ll
end_R1_rocking = end_fb_P_2_ll * rocking_percent * (1 - 2 * a / girder_spacing)

end_fb_M_re = R1_rocking * a

In [110]:
# Wind Force on Loaded Bridge
wind_load = 300 * units('lbf/ft')
end_fb_pws = wind_load*h*floorbeam_spacing/railroad_gage

end_fb_wind_R1 = end_fb_pws * (1-2*a/girder_spacing)

end_fb_M_wl = end_fb_wind_R1 * fb_a

In [111]:
# Required Section Properties and gov moment  # AREMA Table 15-1-11
end_fb_M_tot_1 = sum([total_end_fb_dl_moment, end_fb_M_ll, end_fb_M_imp, end_fb_M_re])
end_fb_M_tot_2 = sum([total_end_fb_dl_moment, end_fb_M_ll, end_fb_M_imp, end_fb_M_re, end_fb_M_wl])/1.25
end_fb_M_tot = max(end_fb_M_tot_1, fb_M_tot_2)

In [112]:
# Governing shear # //TODO - Ensure this isn't need in other calc
end_fb_V_tot_1 = sum([total_end_fb_dl_shear, end_fb_P_2_ll, end_fb_P_imp_ll, end_R1_rocking])
end_fb_V_tot_2 = sum([total_end_fb_dl_shear, end_fb_P_2_ll, end_fb_P_imp_ll, end_R1_rocking, end_fb_wind_R1])/1.25
end_fb_V_tot = max(end_fb_V_tot_1, end_fb_V_tot_2)

In [113]:
total_dead_load
dead_load_per_jack = total_dead_load / 4
jack_pt_offset

3.75 <Unit('foot')>

In [114]:
# Shear and moment doubled to meet requirements of 11-1.8.1
jacking_V_tot = dead_load_per_jack * 2
jacking_M_tot = 2 * dead_load_per_jack * jack_pt_offset

In [115]:
# Required Section Properties

end_fb_gov_M = max(jacking_M_tot, end_fb_M_tot)
end_fb_gov_V = max(jacking_V_tot, end_fb_V_tot)

end_fb_S_x_req = (end_fb_gov_M / (0.55 * F_y)).to('in^3')

In [116]:
# Holes - Assume 4 x 1" holes in web for diaphragm
D1 = 3 * units('in') # //TODO - is this a constant?
D2 = 3 * units('in') + D1

I_holes_web = (diaphragm_num_holes/2*(diaphragm_dia_hole*end_floorbeam.web_thickness)*(D1)**2)+(diaphragm_num_holes/2*(diaphragm_dia_hole*end_floorbeam.web_thickness)*(D2)**2)

In [117]:
I_holes_web

67.5 <Unit('inch ** 4')>

In [118]:
I_holes_flange = lateral_num_holes * lateral_dia_holes * floorbeam.flange_thickness * (floorbeam.depth / 2 - floorbeam.flange_thickness / 2) ** 2

In [119]:
I_net = floorbeam.I_x - I_holes_web - I_holes_flange

In [120]:
I_net

S_x_net = I_net / (floorbeam.depth / 2)

In [121]:
# Allowable Stress (AREMA Table 15-1-12)
Fb_ten = 0.55 * F_y

# Compression in extreme fiber
Fb_comp_1 = 0.55*F_y-(0.55*F_y**2/(6.3*math.pi**2*E_steel))*(diaphragm_spacing_max/floorbeam.r_y)**2
Fb_comp_2 = (0.131 * math.pi * E_steel / (diaphragm_spacing_max * floorbeam.depth * (1+poisson_ratio) ** .5 / (floorbeam.flange_thickness * floorbeam.flange_width))).to('psi')
Fb_comp_3 = 0.55 * F_y

F_b_comp = (min(Fb_comp_3, max(Fb_comp_1, Fb_comp_2))).to('ksi')

f_b_act = (fb_M_tot / S_x_net).to('ksi')

# Bearing area allowable stress
if f_b_act < F_b_comp:
    print(colored(f'OK, Stress Ratio: {f_b_act/F_b_comp}', 'green'))
else:
    print(colored('No Good - Floor Beam Allowable Stress Check', 'red'))

OK, Stress Ratio: 0.3401055219027701 dimensionless


In [122]:
# End Shear
R_dl = total_fb_dl_shear
R_ll = fb_P_ll
R_imp = fb_P_imp_ll * 2
R_re = R1_rocking
R_wl = fb_wind_R1

R_tot = sum([R_dl, R_ll, R_imp, R_re, R_wl])

# Assume 6x1" dia holes in web to connect to girder
fs_net = (R_tot/((floorbeam.depth-web_conn_girder_holes_num*web_conn_girder_holes_dia)*floorbeam.web_thickness)).to('ksi')

F_v = (0.35 * F_y).to('ksi')

# Bearing area allowable stress
if fs_net < F_v:
    print(colored(f'OK, Stress Ratio: {fs_net/F_v}', 'green'))
else:
    print(colored('No Good - Floor Beam End Shear Check', 'red'))

OK, Stress Ratio: 0.2502732273083372 dimensionless


In [123]:
# Bolted Connection - Checked in sep. calc, they account for combined efect so no 1.25 increase
# AREMA 15 1.5.9.a.1 # //TODO - Investigate this standard
R_tot

92465.94656133825 <Unit('force_pound')>

In [124]:
# Floor beam fatigue
fb_M_ll
fb_fat_imp_M = assumed_mean_impact_perc * fb_M_imp
fb_M_fat = fb_M_ll + fb_fat_imp_M

fb_live_load_stress_range = (fb_M_fat / S_x_net).to('ksi')

# N >= 2,000,000 Category B Table 15-1-10 Detail 2.2
fb_fsr = 16 * units('ksi')

# Bearing area allowable stress
if fb_live_load_stress_range < fb_fsr:
    print(colored(f'OK, Stress Ratio: {fb_live_load_stress_range/fb_fsr}', 'green'))
else:
    print(colored('No Good - Floor Beam Fatigue Check', 'red'))

OK, Stress Ratio: 0.32234323685584865 dimensionless


In [125]:
# Deflection
girder_spacing
floorbeam.I_x
E_steel

fb_defl_M = fb_M_ll + fb_M_imp + fb_M_re

fb_w = 8 * fb_defl_M / (girder_spacing ** 2)

fb_deflection = ((5 * fb_w * girder_spacing ** 4) / (384 * E_steel * floorbeam.I_x)).to('in')
max_deflection = (girder_spacing / 640).to('in')

# Deflection Check
if fb_deflection <= max_deflection:
    print(colored(f'OK, Stress Ratio: {fb_deflection/max_deflection}', 'green'))
else:
    print(colored('No Good - Floor Beam Fatigue Check', 'red'))

OK, Stress Ratio: 0.24964506342823886 dimensionless
